In [50]:
import random
import math
from itertools import combinations
import numpy as np
import pylab as pl
from matplotlib import collections as mc
from math import sin, cos, pi, factorial, acos, atan
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from itertools import product, combinations

In [51]:
def dist(a, b):
    """Computes the distance between two points on a sphere"""
    return (sum([(a[i] - b[i]) ** 2 for i in range(len(a))]) ** .5)

def dist_sphere_azim(a, b):
    # a = (r,phi,theta) where theta==lambda longitude
    Ds = acos(sin(a[1])*sin(b[1])+cos(a[1])*cos(b[1])*cos(b[2]-a[2]))
    return (radius*Ds)

def dist_sphere_cart(a, b):
    Ds = acos(sin(atan(a[1]/a[0]))*sin(atan(b[1]/b[0]))+cos(atan(a[1]/a[0]))*cos(atan(b[1]/b[0]))*cos(acos(b[2]/radius)-acos(a[2]/radius)))
    return (radius*Ds)

def dist_sphere_vect(a, b):
    Ds = acos(np.dot(a,b))
    return (radius*Ds)

In [52]:
def unique(lis):
    """determines whether all the elements of a list are unique"""
    return list(set(lis)) == sorted(lis)

In [53]:
def hasNoAlmostPairs_cart(lis):
    """forces trees to not have overlapping steiner points"""
    lx = []
    ly = []
    lz = []
    for i in lis:
        for j in range(len(lx)):
            if abs(i[0]-lx[j]) < .0001 and abs(i[1]-ly[j]) < .0001 and abs(i[2]-ly[j]) < .0001:
                return(False)
        lx.append(i[0])
        ly.append(i[1])
        lz.append(i[2])
    return(True)

In [54]:
a=[[1,1,1],[2,2,2],[1,2,3]]

def hasNoAlmostPairs_sph(lis):
    lr = []
    lphi = []
    ltheta  =[]
    for i in a:
        for j in range(len(lr)):
            if abs(i[0]-lr[j]) < .0001 and abs(i[1]-lphi[j]) < .0001 and abs(i[2]-ltheta[j]) < .0001:
                return(False)
        lr.append(i[0])
        lphi.append(i[1])
        ltheta.append(i[2])
    return(True)

In [55]:
def createTree(snConnections, ssConnections):
    """Finds the tree structure denoted by snConnections (between steiner points
     and nodes) and ssConnections (connections between steiner points and other steiner points"""
    allPoints = [i for i in range(len(snConnections) - 2)] + [i + len(snConnections) - 2 for i in
                                                              range(len(snConnections))]
    tree = [(snConnections[i], i + len(snConnections) - 2) for i in range(len(snConnections))]
    allssConnections = []
    for i in combinations([i for i in range(len(snConnections) - 2)], 2):
        allssConnections.append(i)
    for i in ssConnections:
        tree.append(allssConnections[i])
    return tree

In [57]:
def DFS(tree):
    """Depth-first search"""
    stack = []
    discovered = []
    stack.insert(0, tree[0][0])
#     print('1:',stack)
    while stack:
        v = stack[-1]
        stack = stack[:-1]
#         print('2 stack:',stack)
#         print('2 v:',v)
        if v not in discovered:
            discovered.append(v)
#             print('3 discovered:',discovered)
            for edge in tree:
                if v in edge:
                    for point in edge:
                        if point != v:
                            stack.insert(0, point)
#                             print('5 stack:',stack)
    return discovered

In [58]:
tree = [[1,1,3],[1,3,4],[3,3,5]]
# DFS(tree)

In [59]:
def isFullyAttached(tree):
    if len(DFS(tree)) == len(set.union(set([i[0] for i in tree]), set([i[1] for i in tree]))):
        return True
    return False

In [60]:
isFullyAttached(tree)

False

In [61]:
def isValid(snConnections, ssConnections, steinerPoints):
    #Checks 3 things:
    # 1. whether the graph is a tree (acyclic, connected)
    # 2. whether each steiner point has degree 3
    # 3. there are no overlapping steiner points
    tree = createTree(snConnections, ssConnections)
    if not isFullyAttached(tree) or not hasNoAlmostPairs_cart(steinerPoints):
        return False

    steinerPointsCounts = [0 for i in range(len(snConnections) - 2)]
    for i in snConnections:
        steinerPointsCounts[i] += 1
    allssConnections = []
    for i in combinations([i for i in range(len(snConnections) - 2)], 2):
        allssConnections.append(i)
    for i in ssConnections:
        for j in allssConnections[i]:
            steinerPointsCounts[j] += 1
    if all([i == 3 for i in steinerPointsCounts]):
        return True
    return False

In [62]:
def normalize(vec):
    """converts a vector into a unit vector"""
    #vec = [vec[0]*sin(vec[1])*cos(vec[2]),vec[0]*sin(vec[1])*sin(vec[2]),vec[0]*cos(vec[1])]
    return [i/(sum([j ** 2 for j in vec]) ** .5) for i in vec]

In [63]:
a = [1,2,3]
normalize(a)

[0.2672612419124244, 0.5345224838248488, 0.8017837257372732]

In [64]:
def nCr(n, r):
    if n < r:
        return 0
    return factorial(n) / (factorial(r) * factorial(n - r))

In [65]:
def pathLength(nodes, steinerPoints, ssConnections, snConnections):
    """computes the length of the current tree"""
    n = len(nodes)
    d = 0
    d += sum([dist(nodes[i], steinerPoints[snConnections[i]]) for i in range(n)])
    # allssConnections = []
    # for i in combinations([i for i in range(n - 2)], 2):
    #     allssConnections.append(i)
    allssConnections = [j for j in combinations([i for i in range(n - 2)], 2)]
    d += sum([dist(steinerPoints[allssConnections[i][0]],
              steinerPoints[allssConnections[i][1]]) for i in ssConnections])
    return d


In [66]:
def steinerTree(nodes):
    """Computes the Steiner Tree using an iterative heuristic"""
    #works in 2 or 3 dimensions
    R = len(nodes[0]) # either 2 or 3 -- this is the dimension we're working in
    n = len(nodes)
    steinerPoints = []
    for i in range(n - 2):
        steinerPoints.append([random.uniform(min([i[dim] for i in nodes]), max([i[dim] for i in nodes])) for dim in
                     range(R)])
    jump = 0
    for i in steinerPoints:
        for j in nodes:
            jump += dist(i, j)
    jump /= (len(steinerPoints) * len(nodes))
    #now the initial topology must be created
    snLocs = [i for i in range(n - 2)]
    snConnections = [random.choice(snLocs) for i in range(len(nodes))] #connections between steiner points and nodes
    ssLocs = [i for i in range(int(nCr(len(steinerPoints), 2)))]
    ssConnections = []  #connections between steiner points and other steiner points
    for i in range(n - 3):
        ssConnections.append(random.choice(ssLocs))
        ssLocs.remove(ssConnections[-1])
    print(createTree(snConnections, ssConnections))  #this is the structure of the initial tree
    iterations = 0
    while iterations < 25000:
        oldConnections = (snConnections[:],
                          ssConnections[:])  #these fucking colons needing to be here cost me hours of time

        vec = [random.random() for dim in range(R)]
        negaters = [random.randint(0, 1) for dim in range(R)]
        for dim in range(R):
            if negaters[dim]:
                vec[dim] *= -1
        vec = normalize(vec)
        #multiply each component by the jump size
        for j in range(R):
            vec[j] *= jump
        r = random.randint(0, len(steinerPoints) - 1)
        newsol = [steinerPoints[r][dim] + vec[dim] for dim in range(R)]
        newsteinerPoints = steinerPoints[:r] + [newsol] + steinerPoints[r + 1:]
        if pathLength(nodes, newsteinerPoints, ssConnections, snConnections) < \
                pathLength(nodes, steinerPoints, ssConnections, snConnections):
            steinerPoints = newsteinerPoints

        r1 = random.randint(0, len(snConnections) - 1)
        r2 = random.randint(0, len(snConnections) - 1)
        newSnConnections = snConnections[:]
        newSnConnections[r1], newSnConnections[r2] = newSnConnections[r2], newSnConnections[r1]
        if pathLength(nodes, steinerPoints, ssConnections, newSnConnections) < \
                pathLength(nodes, steinerPoints, ssConnections,snConnections):
            snConnections = newSnConnections[:]
        r = random.randint(0, len(ssConnections) - 1)
        newSsConnection = random.randint(0, nCr(len(steinerPoints), 2) - 1)
        if pathLength(nodes, steinerPoints, ssConnections[:r] + [newSsConnection] + ssConnections[r + 1:], snConnections) < \
                pathLength(nodes, steinerPoints, ssConnections, snConnections) and unique(
                                ssConnections[:r] + [newSsConnection] + ssConnections[r + 1:]):
            ssConnections[r] = newSsConnection
            allssConnections = [i for i in combinations([i for i in range(n - 2)], 2)]
            steinerPointsCounts = [3 for i in range(len(steinerPoints))]
            for i in ssConnections:
                for j in allssConnections[i]:
                    steinerPointsCounts[j] -= 1
            snConnections = []
            for i in range(len(steinerPointsCounts)):
                for j in range(steinerPointsCounts[i]):
                    snConnections.append(i)
            random.shuffle(snConnections)
        if not isValid(snConnections, ssConnections, steinerPoints):
            snConnections, ssConnections = oldConnections
        jump *= .9995
        iterations += 1
        if iterations == 25000 and not isValid(snConnections, ssConnections, steinerPoints):
            # restarts if we've failed
            print("Starting over...")
            steinerPoints = []
            for i in range(n - 2):
                steinerPoints.append([random.uniform(min([i[dim] for i in nodes]), max([i[dim] for i in nodes])) for dim in
                             range(R)])
            jump = 0
            for i in steinerPoints:
                for j in nodes:
                    jump += dist(i, j)
            jump /= (len(steinerPoints) * len(nodes))
            #now the initial topology must be created
            snLocs = [i for i in range(n - 2)]
            snConnections = [random.choice(snLocs) for i in range(len(nodes))] #connections between steiner points and nodes
            ssLocs = [i for i in range(int(nCr(len(steinerPoints), 2)))]
            ssConnections = []  #connections between steiner points and other steiner points
            for i in range(n - 3):
                ssConnections.append(random.choice(ssLocs))
                ssLocs.remove(ssConnections[-1])
            iterations = 0

    #wrap up program

    print("steinerPoints:")
    for sol in steinerPoints:
        print(sol)
    print("ssConnections: ", ssConnections)
    print("snConnections: ", snConnections)
    print("tree: ", createTree(snConnections, ssConnections))
    print(pathLength(nodes, steinerPoints, ssConnections, snConnections))
    # if not isValid(snConnections, ssConnections):
    #     print("I have not generated a valid Steiner tree for you. I am very sorry.")
    #     return

    #for 3D plots
    if R == 3:
        lines = []
        for i in range(n):
            lines.append([nodes[i], steinerPoints[snConnections[i]]])
        allssConnections = []
        for i in combinations([i for i in range(n - 2)], 2):
            allssConnections.append(i)
        for i in ssConnections:
            lines.append([steinerPoints[allssConnections[i][0]], steinerPoints[allssConnections[i][1]]])
        VecStart_x = []
        VecStart_y = []
        VecStart_z = []
        VecEnd_x = []
        VecEnd_y = []
        VecEnd_z = []
        for line in lines:
            VecStart_x.append(line[0][0])
            VecEnd_x.append(line[1][0])
            VecStart_y.append(line[0][1])
            VecEnd_y.append(line[1][1])
            VecStart_z.append(line[0][2])
            VecEnd_z.append(line[1][2])

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        for i in range(len(VecStart_x)):
            ax.plot([VecStart_x[i], VecEnd_x[i]], [VecStart_y[i], VecEnd_y[i]], zs=[VecStart_z[i], VecEnd_z[i]])
        pl.plot([i[0] for i in steinerPoints], [i[1] for i in steinerPoints], [i[2] for i in steinerPoints], 'bo')
        pl.plot([i[0] for i in nodes], [i[1] for i in nodes], [i[2] for i in nodes], 'ro')
        # ax.text(min([i[0] for i in nodes])-1, min(i[1] for i in nodes)-1, min(i[2] for i in nodes)-1,
        #         "Total distance: "+str(pathLength(nodes, steinerPoints, ssConnections, snConnections)), fontsize=15)
        ax.set_title("Total Distance: " + str(pathLength(nodes, steinerPoints, ssConnections, snConnections)))

        ## draw sphere
#        u = np.linspace(0, 2 * np.pi, 100)
#        v = np.linspace(0, np.pi, 100)
#
#        x = 1 * np.outer(np.cos(u), np.sin(v))
#        y = 1 * np.outer(np.sin(u), np.sin(v))
#        z = 1 * np.outer(np.ones(np.size(u)), np.cos(v))
#        elev = 10.0
#        rot = 80.0 / 180 * np.pi
#        ax.plot_surface(x, y, z,  rstride=4, cstride=4, color='b', linewidth=0, alpha=0.5)
#        pl.show()
        # Create a sphere
        pi = np.pi
        cos = np.cos
        sin = np.sin
        phi, theta = np.mgrid[0.0:pi:100j, 0.0:2.0*pi:100j]
        x = radius*sin(phi)*cos(theta)
        y = radius*sin(phi)*sin(theta)
        z = radius*cos(phi)


        def slerp(p1, p2, t):
            omega = np.arccos( p1.dot(p2) )
            sin_omega = np.sin(omega)
            t = t[:, np.newaxis]
            return ( np.sin( (1-t)*omega )*p1 + np.sin( t*omega )*p2 )/sin_omega

        p1 = np.array([1, 0, 0])
        p2 = np.array([0, 1, 0])
        t = np.linspace(0, 1, 30)

        arc = slerp(p1, p2, t)

         #Import data
#        data = np.genfromtxt('leb.txt')
#        theta, phi, r = np.hsplit(data, 3)
#        theta = theta * pi / 180.0
#        phi = phi * pi / 180.0
#        xx = sin(phi)*cos(theta)
#        yy = sin(phi)*sin(theta)
#        zz = cos(phi)

        #Set colours and render
#        ax = fig.add_subplot(111, projection='3d')

        ax.plot_surface(
            x, y, z,  rstride=1, cstride=1, color='c', alpha=0.3, linewidth=0)

        pl.plot( arc[:, 0], arc[:, 1] )
        ax.set_xlim([-1,1])
        ax.set_ylim([-1,1])
        ax.set_zlim([-1,1])
#        ax.set_aspect("equal")
        pl.tight_layout()
        manager = plt.get_current_fig_manager()
        manager.window.showMaximized()
        plt.savefig('Steiner_tree.png')
        pl.show()


In [69]:
nodes = [(1, -1.5158748173898942, 6.149991850504285),
 (1, 0.04114338066861693, 5.321074040934245),
 (1, 0.15580614823217237, 4.354108193031286),
 (1, -0.4753753803709867, 5.763061153369418)]

R = len(nodes[0]) # either 2 or 3 -- this is the dimension we're working in
n = len(nodes)
steinerPoints = []
for i in range(n - 2):
    steinerPoints.append([random.uniform(min([i[dim] for i in nodes]), max([i[dim] for i in nodes])) for dim in
                 range(R)])
jump = 0
for i in steinerPoints:
    for j in nodes:
        jump += dist(i, j)
jump /= (len(steinerPoints) * len(nodes))
#now the initial topology must be created
snLocs = [i for i in range(n - 2)]
snConnections = [random.choice(snLocs) for i in range(len(nodes))] #connections between steiner points and nodes
ssLocs = [i for i in range(int(nCr(len(steinerPoints), 2)))]
ssConnections = []  #connections between steiner points and other steiner points
for i in range(n - 3):
    ssConnections.append(random.choice(ssLocs))
    ssLocs.remove(ssConnections[-1])
print(createTree(snConnections, ssConnections))  #this is the structure of the initial tree

[(0, 2), (0, 3), (1, 4), (1, 5), (0, 1)]


In [68]:
#6 points on a sphere
n=4
radius = 1
nodes = []
for i in range(n):
    phi = np.pi*random.uniform(-1,1)/2
    theta = 2*np.pi*random.uniform(0,1)
#     x = radius*sin(th)*cos(phi)
#     y = radius*sin(th)*sin(phi)
#     z = radius*cos(th)
    nodes.append((radius,phi,theta))
#print(dist(nodes[0],nodes[1]))
# steinerTree(nodes)
nodes

[(1, -1.5158748173898942, 6.149991850504285),
 (1, 0.04114338066861693, 5.321074040934245),
 (1, 0.15580614823217237, 4.354108193031286),
 (1, -0.4753753803709867, 5.763061153369418)]